In [ ]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

FILE = "../data/filtered/06_stemmed_text.parquet"
OUT_FILE = "../data/filtered/07_sampled_news.parquet"
N = 10
SEED = 275877

In [ ]:
np.random.seed(SEED)

lf = pl.scan_parquet(FILE)
lf_height = lf.select(pl.len()).collect().item()
lf = (
    lf.with_columns(
        pl.lit(np.random.rand(lf_height)).alias("rand")
    )
    .sort(["date", "rand"])
    .group_by("date")
    .head(N)
    .select([col for col in lf.collect_schema().names()])
)
df = lf.collect()
df.write_parquet(OUT_FILE)
df

In [ ]:
df = (
    df 
    .group_by(pl.col("date"))
    .agg(
        pl.col("url").count().alias("num_urls"),
        pl.col("title").count().alias("num_titles"),
        pl.col("text").count().alias("num_articles"),
    )
    .sort("date")
)
df

In [ ]:
plt.figure(figsize=(14,5))
fig, ax = plt.subplots()
ax.plot(df["date"].to_numpy(), df["num_articles"].to_numpy())
plt.show()